In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import numpy as np
from joblib import dump
from src.models import eval_model as evm
from src.models import eval_baseline as evb
from sklearn.utils import resample
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("../data/processed/df_cleaned_nba_prediction.csv")

In [3]:
df

,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,OREB,...,REB,AST,STL,BLK,TOV,TARGET_5Yrs,CALC3P%,CALCFG%,CALCFT%,TARGET_5Yrs_Inv
0,80,24.3,7.8,3.0,6.4,0.1,0.3,2.0,2.9,2.2,...,3.8,3.2,1.1,0.2,1.6,1,33.333333,46.875000,68.965517,0
1,75,21.8,10.5,4.2,7.9,0.0,0.0,2.4,3.6,3.6,...,6.6,0.7,0.5,0.6,1.4,1,0.000000,53.164557,66.666667,0
2,85,19.1,4.5,1.9,4.5,0.4,1.2,0.4,0.6,0.6,...,2.4,0.8,0.4,0.2,0.6,1,33.333333,42.222222,66.666667,0
3,63,19.1,8.2,3.5,6.7,0.3,0.8,0.9,1.5,0.8,...,3.0,1.8,0.4,0.1,1.9,1,37.500000,52.238806,60.000000,0
4,63,17.8,3.7,1.7,3.4,0.5,1.4,0.2,0.5,2.4,...,4.9,0.4,0.4,0.6,0.7,1,35.714286,50.000000,40.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,32,9.2,1.8,0.7,1.8,0.0,0.0,0.4,0.6,0.3,...,1.9,0.5,0.3,0.2,0.4,1,0.000000,38.888889,66.666667,0
7996,54,6.0,1.8,0.7,1.4,0.1,0.1,0.2,0.4,1.0,...,2.0,0.1,0.0,0.3,0.3,1,100.000000,50.000000,50.000000,0
7997,85,28.2,10.7,4.0,9.0,0.2,0.6,2.8,3.9,1.0,...,3.1,3.4,1.2,0.2,1.8,1,33.333333,44.444444,71.794872,0
7998,39,7.7,2.5,1.0,2.3,0.0,0.0,0.6,0.7,0.4,...,0.9,0.2,0.3,0.3,0.5,1,0.000000,43.478261,85.714286,0


In [4]:
df_minority = df[df['TARGET_5Yrs_Inv']==1]
df_majority = df[df['TARGET_5Yrs_Inv']==0]

In [5]:
[df_minority.shape , df_majority.shape]

[(1331, 21), (6669, 21)]

In [6]:
df_minority['TARGET_5Yrs_Inv'].value_counts(), df_majority['TARGET_5Yrs_Inv'].value_counts()

(1    1331
 Name: TARGET_5Yrs_Inv, dtype: int64,
 0    6669
 Name: TARGET_5Yrs_Inv, dtype: int64)

In [7]:
df_minority_upsampled = resample(df_minority,replace=True,n_samples=6669,random_state=123) 
df_majority_downsampled = resample(df_majority,replace=True,n_samples=1331,random_state=123) 

In [8]:
[df_minority_upsampled.shape , df_majority_downsampled.shape]

[(6669, 21), (1331, 21)]

In [9]:
df_minority_upsampled['TARGET_5Yrs_Inv'].value_counts(), df_majority_downsampled['TARGET_5Yrs_Inv'].value_counts()

(1    6669
 Name: TARGET_5Yrs_Inv, dtype: int64,
 0    1331
 Name: TARGET_5Yrs_Inv, dtype: int64)

#### Save Data as CSV for processing

In [10]:
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
df_upsampled[['TARGET_5Yrs','TARGET_5Yrs_Inv']].value_counts()

TARGET_5Yrs  TARGET_5Yrs_Inv
0            1                  6669
1            0                  6669
dtype: int64

In [11]:
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_downsampled[['TARGET_5Yrs','TARGET_5Yrs_Inv']].value_counts()

TARGET_5Yrs  TARGET_5Yrs_Inv
0            1                  1331
1            0                  1331
dtype: int64

In [12]:
df_upsampled.to_csv('../data/processed/df_cleaned_upsampled_nba_prediction.csv',index=False)
df_downsampled.to_csv('../data/processed/df_cleaned_downsampled_nba_prediction.csv',index=False)